In [18]:
import pandas as pd

import string
import nltk
import spacy

from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [2]:
df_isp_irc = pd.read_excel('vt_isp_irc.xlsx')
df_isp_task = pd.read_excel('vt_isp_task_mhrs.xlsx')
df_isp_material = pd.read_excel('vt_isp_material_consumption_pricing.xlsx')

df_isq_irc = pd.read_excel('vt_isq_irc.xlsx')
df_isq_task = pd.read_excel('vt_isq_task_mhrs.xlsx')
df_isq_material = pd.read_excel('vt_isq_material_consumption_pricing.xlsx')

df_iux_irc = pd.read_excel('vt_iux_irc.xlsx')
df_iux_task = pd.read_excel('vt_iux_task_mhrs.xlsx')
df_iux_material = pd.read_excel('vt_iux_material_consumption_pricing.xlsx')

In [3]:
df_isp_irc.head(2)

,DeleteFlag,Type,Log Item #,ATA #,Description,Corrective Action,Discrepancy #,Action,Status,Source Task/Discrep. #,...,Deferral FC,Deferral Parameter,Deferral Value,Deferral Calendar Value,Deferral Calendar,Deferral Item #,Deferral Type,Reason for Deferral,Auth. Ref. #,Message Center
0,No,MIREP,HMV23/000059/0923/1,53,1. DURING INSPECTION OBERVED LH WING SLATE #01...,1. CARRIED OUT INSTALLATION OF LH WING SLAT #0...,HMV23/000059/0923/1,Close,Closed,AIRCRAFT ARRIVAL INSPECTION,...,NaN,NaN,NaN,NaN,Hours,NaN,NaN,NaN,NaN,NaN
1,No,MIREP,HMV23/000059/0923/2,34,DURING ARRIVAL INSPECTION CHECK FOUND FAILURE ...,CARRIED OUT TROUBLESHOOTING IAW TSM TASK 31-32...,HMV23/000059/0923/2,Close,Closed,AIRCRAFT ARRIVAL INSPECTION,...,NaN,NaN,NaN,NaN,Hours,NaN,NaN,NaN,NaN,NaN


### Fields to consider in 'VT ISP IRC'
- Planned Start Date
- Planned End Date
- Actual Start Date
- Actual End Date
- Est. Man Hrs.
- Actual Man Hrs.
- Insp Req?
- Mech Req?
- Part Required?
- Contract Classification
- Source Task/Discrep. #
- Description
- Corrective Action
- Log Item #

In [6]:
df_isq_irc.head(2)

,DeleteFlag,Type,Log Item #,ATA #,Description,Corrective Action,Discrepancy #,Action,Status,Source Task/Discrep. #,...,Deferral FC,Deferral Parameter,Deferral Value,Deferral Calendar Value,Deferral Calendar,Deferral Item #,Deferral Type,Reason for Deferral,Auth. Ref. #,Message Center
0,No,MIREP,HMV23/000055/0923/1,57,"DURING ARRIVAL INSPECTION , OBSERVED DENT ON S...",REFER SDIRC HMV23/000055/0923/7 FOR REPAIR DET...,HMV23/000055/0923/1,Close,Closed,AIRCRAFT ARRIVAL INSPECTION,...,NaN,NaN,NaN,NaN,Hours,NaN,NaN,NaN,NaN,NaN
1,No,MIREP,HMV23/000055/0923/2,53,DURING INSPECTION OBSERVED LH WING SLAT#5 WEAT...,CARRIED OUT INSTALLATION OF LH WING SLAT #5 OU...,HMV23/000055/0923/2,Close,Closed,AIRCRAFT ARRIVAL INSPECTION,...,NaN,NaN,NaN,NaN,Hours,NaN,NaN,NaN,NaN,NaN


In [7]:
df_iux_irc.head(2)

,DeleteFlag,Type,Log Item #,ATA #,Description,Corrective Action,Discrepancy #,Action,Status,Source Task/Discrep. #,...,Deferral FC,Deferral Parameter,Deferral Value,Deferral Calendar Value,Deferral Calendar,Deferral Item #,Deferral Type,Reason for Deferral,Auth. Ref. #,Message Center
0,No,MIREP,HMV23/000043/0823/1,33,DURING ARRIVAL INSPECTION OBSERVED \n1. RH NAV...,1. RH NAV POSITION LIGHT TROUBLESHOOTING CARRI...,HMV23/000043/0823/1,Close,Closed,AIRCRAFT ARRIVAL INSPECTION,...,NaN,NaN,NaN,NaN,Hours,NaN,NaN,NaN,NaN,NaN
1,No,MIREP,HMV23/000043/0823/2,25,CARRY OUT CA SEATS AND DOOR TRIMS (QTY49) PAIN...,1.FWD AND AFT WALL MOUNTED CABIN ATTENDENT SEA...,HMV23/000043/0823/2,Close,Closed,AWR23/002577/0823,...,NaN,NaN,NaN,NaN,Hours,NaN,NaN,NaN,NaN,NaN


### Tokenize and embed the descriptions of unexpected observations made.

In [9]:
# Download NLTK Resources
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /home/ranjith/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/ranjith/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /home/ranjith/nltk_data...


True

In [10]:
# Load spacy en_core_web_sm model
nlp = spacy.load("en_core_web_sm")

In [26]:
def preprocess_text(text: str, preserve_symbols=[]) -> str:
    '''
    This function performs text preprocessing and returns processed text. 
    It will also accept a list of symbols to preserve.
    Input: text
    Output: text
    '''
    # Define symbols to preserve
    preserve_symbols = set(preserve_symbols)
    
    # Remove punctuation, excluding specified symbols
    custom_translation = str.maketrans('', '', ''.join(set(string.punctuation) - preserve_symbols))
    text = text.translate(custom_translation)
    return text


def tokenization(preprocessed_text: str) -> list:
    '''
    This function performs text tokenization and returns a list of tokens.
    Input: text
    Output: list of tokens
    '''
    sentences = sent_tokenize(preprocessed_text)
    preprocessed_tokens = []

    # Tokenization and stopword removal
    for sentence in sentences:
        tokens = word_tokenize(sentence)
        stop_words = set(stopwords.words('english'))
        tokens = [token for token in tokens if token.lower() not in stop_words]
        
        # Lemmatization
        lemmatizer = WordNetLemmatizer()
        tokens = [lemmatizer.lemmatize(token) for token in tokens]

        preprocessed_tokens.append(tokens)

    return preprocessed_tokens


def calculate_embeddings(preprocessed_tokens: list) -> list:
    '''
    This function performs text embedding and returns a list of embeddings.
    Input: list of tokens
    Output: list of embeddings
    '''
    # Embedding
    embeddings = []
    for token in preprocessed_tokens:
        sentence = ' '.join(token)
        doc = nlp(sentence)
        sentence_embedding = doc.vector
        embeddings.append(sentence_embedding)

    return embeddings


def main(text: str, preserve_symbols=['#', '/', '-', ':']) -> list:
    '''
    This function performs text preprocessing, tokenization, embedding and returns a list of embeddings.
    Input: text
    Output: list of embeddings
    '''
    preprocessed_text = preprocess_text(text, preserve_symbols)
    preprocessed_tokens = tokenization(preprocessed_text)
    embeddings = calculate_embeddings(preprocessed_tokens)

    return embeddings

In [27]:
df_isp_irc.groupby(by='Source Task/Discrep. #').get_group('AIRCRAFT ARRIVAL INSPECTION')['Description'].tolist()

['1. DURING INSPECTION OBERVED LH WING SLATE #01, TRACK#02 ,#03, #04 UPPER BULB SEAL DAMAGE.SAME TO BE REPLACED.\n\n2. DURING INSPECTION OBSERVED SLAT#02 TRACK #5 PLATE DAMAGE.SAME TO BE REPLACED.',
 'DURING ARRIVAL INSPECTION CHECK FOUND FAILURE MESSAGE IN PFR " NO ADF 1 DATA (INTM) ".',
 'DURING ARRIVAL INSPECTION FOUND STATIC DISCHARGERS DAMAGED AT LOCATION : \n1) LH HORZ STAB TYPE A DISCHARGER QTY# 2 & TYPE B DISCHARGER QTY#3 \n2) RH HORZ STAB TYPE A DISCHARGER QTY#1 \n3)VERTICAL STABILISER TYPE A DISCHARGER QTY 01 & TYPE B DISCHARGER QTY 02.',
 'DURING ARRIVAL CHECK FOUND SERVICE LIGHT LAMP INOP AT FOLLOWING LOCATION \n1.103VU BATTERY COMPARMENT LAMP FIN: 8LS QTY#1,.\n2. REFUEL/DEFUEL CONTROL PANEL FLOODLIGHT LAMP FIN: 32QU.',
 'DURING ARRIVAL INSPECTION , OBSERVED FOLLOWINS DICREPANCIES.\n\n1) RH SIDE WING NO#1 SLAT, NO.3 TRACK SQURE SEAL UPPER BULB SEAL DEGRADED.\n2) RH SIIDE WING SLAT NO#5 OUT BOARD WEATHER SEAL ERODED. SAME TO BE REPLACED.',
 'DURING INSPECTION FOUND FOLLOWING

In [44]:
grouped_df_Description = df_isp_irc.groupby(by='Source Task/Discrep. #')['Description'].apply(lambda x: x.apply(main))
grouped_df_Corrective_Action = df_isp_irc.groupby(by='Source Task/Discrep. #')['Corrective Action'].apply(lambda x: x.apply(main))
df_isp_irc['description_correction'] = df_isp_irc['Description'] +' '+df_isp_irc['Corrective Action']
grouped_df_description_correction = df_isp_irc.groupby(by='Source Task/Discrep. #')['description_correction'].apply(lambda x: x.apply(main))

In [45]:
grouped_df_Description

Source Task/Discrep. #         
215222-01-1 (LH)             9     [[-0.404925, -0.054837763, 0.015778754, -0.463...
                             12    [[-0.6733928, -0.17803305, 0.13669693, 0.15644...
252100-01-1                  16    [[-0.5620832, -0.006267702, -0.22605507, 0.423...
384241-01-1                  11    [[-0.22561774, -0.09848608, 0.3168459, 0.54762...
531900-03-1                  10    [[-0.5104654, -0.32680652, 0.36707053, 0.03859...
AIRCRAFT ARRIVAL INSPECTION  0     [[-0.64320844, -0.07742605, -0.32219216, 0.279...
                             1     [[-0.34986612, -0.25470546, -0.20057899, 0.184...
                             2     [[-0.25188813, -0.16425797, -0.39383945, 0.249...
                             3     [[-0.55938756, -0.23792706, -0.15409206, 0.058...
                             4     [[-0.44807178, -0.30468988, -0.2933644, 0.2563...
                             5     [[-0.3829494, -0.45530254, 0.28193352, 0.19516...
                             6   

In [46]:
df_isp_irc['description_embedding'] = grouped_df_Description.explode().tolist()
df_isp_irc['corrective_action_embedding'] = grouped_df_Corrective_Action.explode().tolist()
df_isp_irc['description_correction_embedding'] = grouped_df_description_correction.explode().tolist()

In [49]:
df_isp_irc[['Source Task/Discrep. #', 'Description', 'Corrective Action', 'description_embedding', 'corrective_action_embedding', 'description_correction_embedding']].head()

,Source Task/Discrep. #,Description,Corrective Action,description_embedding,corrective_action_embedding,description_correction_embedding
0,AIRCRAFT ARRIVAL INSPECTION,1. DURING INSPECTION OBERVED LH WING SLATE #01...,1. CARRIED OUT INSTALLATION OF LH WING SLAT #0...,"[-0.404925, -0.054837763, 0.015778754, -0.4631...","[-0.7883508, -0.48455766, 0.053018823, 0.28613...","[-0.64392567, -0.32557, 0.09585165, 0.09748258..."
1,AIRCRAFT ARRIVAL INSPECTION,DURING ARRIVAL INSPECTION CHECK FOUND FAILURE ...,CARRIED OUT TROUBLESHOOTING IAW TSM TASK 31-32...,"[-0.6733928, -0.17803305, 0.13669693, 0.156445...","[-0.81899613, -0.536668, 0.20998687, 0.4754251...","[-0.7522026, -0.45083523, 0.18606621, 0.415954..."
2,AIRCRAFT ARRIVAL INSPECTION,DURING ARRIVAL INSPECTION FOUND STATIC DISCHAR...,STATIC DISCHARGERS REPLACED AT FOLLOWING LOCAT...,"[-0.5620832, -0.006267702, -0.22605507, 0.4239...","[-0.6431541, -0.4975288, 0.35840395, 0.6532554...","[-0.6112287, -0.3528504, 0.17231278, 0.5829198..."
3,AIRCRAFT ARRIVAL INSPECTION,DURING ARRIVAL CHECK FOUND SERVICE LIGHT LAMP ...,1. CARRIED OUT REPLACEMENT OF SERVICE LIGHT LA...,"[-0.22561774, -0.09848608, 0.3168459, 0.547622...","[-0.7275441, -0.58689135, 0.39343697, 0.385342...","[-0.57389027, -0.44901672, 0.30712327, 0.43411..."
4,AIRCRAFT ARRIVAL INSPECTION,"DURING ARRIVAL INSPECTION , OBSERVED FOLLOWINS...","1) RH SIDE WING NO#1 SLAT, NO.3 TRACK SQURE SE...","[-0.5104654, -0.32680652, 0.36707053, 0.038590...","[-0.7171926, -0.6179478, 0.64386773, 0.5765455...","[-0.63411736, -0.52430564, 0.5837834, 0.416250..."
